<a href="https://colab.research.google.com/github/Adhini99/Fine_Tuning_Bert/blob/main/Fine_Tuning_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install -q datasets scikit-learn evaluate

In [ ]:
!pip install -U transformers

In [ ]:
# Import required libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

In [ ]:
# 1️⃣ Load Dataset
dataset_name = "ag_news"
dataset = load_dataset(dataset_name)
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [ ]:
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
# 2️⃣ Load tokenizer
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# Example input string
text = "The stock market saw record gains today."

# Tokenize (convert to input IDs and attention mask)
inputs = tokenizer(text, return_tensors="pt")

# Show tokenized input
print("📥 Tokenized input:")
for k, v in inputs.items():
    print(f"{k}: {v.shape} -> {v}")


📥 Tokenized input:
input_ids: torch.Size([1, 10]) -> tensor([[  101,  1996,  4518,  3006,  2387,  2501, 12154,  2651,  1012,   102]])
token_type_ids: torch.Size([1, 10]) -> tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask: torch.Size([1, 10]) -> tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
# 3️⃣ Tokenize data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
# Load model
num_labels = dataset["train"].features["label"].num_classes
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


for param in model.base_model.parameters():
    param.requires_grad = False #768*4 params train

# Unfreeze only the pooler layer
for param in model.bert.pooler.parameters():
    param.requires_grad = True

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {
        "Total": total_params,
        "Trainable": trainable_params,
        "Frozen": total_params - trainable_params
    }

# Example:
param_counts = count_parameters(model)
print("Model Parameter Counts:")
for k, v in param_counts.items():
    print(f"{k}: {v:,}")


Model Parameter Counts:
Total: 109,485,316
Trainable: 593,668
Frozen: 108,891,648


In [ ]:
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1835,  0.5588, -0.5476,  0.4933]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
import evaluate
import numpy as np

# Load the evaluation metric
accuracy_metric = evaluate.load("accuracy")

# Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# 6️⃣ Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",           # 🔁 Log training loss every N steps
    logging_steps=10,                   # ⏱ Logs every 10 steps
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)


In [ ]:
# 7️⃣ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2112826755.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 8️⃣ Train model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.285800,0.313896,0.891974
2,0.312600,0.302188,0.894605
3,0.346500,0.299458,0.896184


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=22500, training_loss=0.33649615867932636, metrics={'train_runtime': 2848.4489, 'train_samples_per_second': 126.385, 'train_steps_per_second': 7.899, 'total_flos': 9.472168083456e+16, 'train_loss': 0.33649615867932636, 'epoch': 3.0})

In [ ]:
label_map = dataset["train"].features["label"].int2str  # maps 0 → 'World', etc.

In [ ]:
import torch

def classify(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs={k:v.to(model.device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax(dim=-1).item()
    return label_map(predicted_class_id)

In [ ]:
print(classify("The stock market saw record gains today."))       # → 'Business'

Business
